In [ ]:
from google.colab import drive
drive.mount('/gdrive')

!pip install catboost 
!pip install tensorflow-addons==0.15.0

# !pip install pycaret[full]
# from pycaret.utils import enable_colab
# from pycaret.classification import *

import os
import pandas as pd
import numpy as np
import sklearn.decomposition

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold , KFold
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras import layers 
import tensorflow_addons.metrics

In [ ]:
path="/gdrive/My Drive/Colab Notebooks/dacon/dna_classify"
data_folder = f"{path}/data"

train = pd.read_csv(f"{data_folder}/train.csv")
test = pd.read_csv(f"{data_folder}/test.csv")
sample_submission = pd.read_csv(f"{data_folder}/sample_submission.csv")

display(train,test)

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,TEST_000,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,TEST_001,0,0,0,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,TEST_002,0,0,0,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,TEST_003,0,0,0,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,TEST_004,0,0,0,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,TEST_170,0,0,0,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A
171,TEST_171,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A
172,TEST_172,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G
173,TEST_173,0,0,0,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A


In [ ]:
for col in train:
    print("----------",col,"----------")
    print("train")
    print(train[col].value_counts())
    print("test")
    print(test[col].value_counts())

---------- id ----------
train
TRAIN_000    1
TRAIN_164    1
TRAIN_166    1
TRAIN_167    1
TRAIN_168    1
            ..
TRAIN_092    1
TRAIN_093    1
TRAIN_094    1
TRAIN_095    1
TRAIN_261    1
Name: id, Length: 262, dtype: int64
test
TEST_000    1
TEST_120    1
TEST_112    1
TEST_113    1
TEST_114    1
           ..
TEST_060    1
TEST_061    1
TEST_062    1
TEST_063    1
TEST_174    1
Name: id, Length: 175, dtype: int64
---------- father ----------
train
0    262
Name: father, dtype: int64
test
0    175
Name: father, dtype: int64
---------- mother ----------
train
0    262
Name: mother, dtype: int64
test
0    175
Name: mother, dtype: int64
---------- gender ----------
train
0    262
Name: gender, dtype: int64
test
0    175
Name: gender, dtype: int64
---------- trait ----------
train
2    193
1     69
Name: trait, dtype: int64
test
2    124
1     51
Name: trait, dtype: int64
---------- SNP_01 ----------
train
G G    141
A G     71
A A     50
Name: SNP_01, dtype: int64
test
G G    84


KeyError: ignored

In [ ]:
class Preprocess:
    train_x = None
    test_x = None
    test_x = None
    snp_cols = {"SNP_"+str(i).zfill(2):None for i in range(1,16)}

    def __init__(self, train, test):
        train_df = train
        test_df = test

        self.train_x = train_df.drop(columns=["id","father","mother","gender"])
        self.test_x = test_df.drop(columns=["id","father","mother","gender"])

        self.train_y = self.train_x["class"]
        self.train_x = self.train_x.drop(columns=["class"])

        # encoder_dict = self.do_label_encoding(self.train_x,self.snp_cols)
        # self.test_x = self.labeling(self.test_x,encoder_dict)

    def do_label_encoding(self,df,encoder_dict):
        for col in encoder_dict:
            le=LabelEncoder()
            le.fit(df[col])
            
            encoder_dict[col]=le
            df[col] = list(le.transform(df[col]))

        return df, encoder_dict

    def labeling(self,df,encoder_dict):
        for key,encoder in encoder_dict.items():
            df[key] = encoder.transform(df[key])
        return df

    def nominal_onehot(self,df,cols):
        # make nominal onehot
        for col in cols:
            # dummy
            dummy_df = pd.get_dummies(df[col])
            # delete origin col
            df = df.drop([col],axis=1)
            # change col name type {origin col name}:dummy:{value}
            dummy_df.columns = [col+":dummy:"+str(val) for val in dummy_df.columns]
            # concat 
            df = pd.concat([df,dummy_df],axis=1)
        return df
    def split_alpha(self,df,cols):
        '''
        cols 컬럼들의 알파벳을 두개로 쪼갠다.
        '''
        for col in cols:
            df[col+"_01"] = df[col].str[0]
            df[col+"_02"] = df[col].str[2]
        
            df = df.drop(columns=[col])
        
        return df

    def add_order(self,df):
        '''
        순서를 넣는다
        '''
        size = len(df)
        order_df = pd.DataFrame({"order":[i for i in range(size)]})
        
        return pd.concat([order_df,df],axis=1)

    def same_col(self,df):
        for col in self.snp_cols:
            df[col+"_is_same"] = np.array(df[col].str[0]==df[col].str[2],dtype=np.int64)
        return df

In [ ]:
class MyModel:
    train_x = None
    train_y = None
    test_x = None
    
    model = None
    col_size = None
    sample_submission = pd.read_csv(f"{data_folder}/sample_submission.csv")

    def __init__(self,train_x,train_y,test_x):
        self.train_x = train_x
        self.train_y = train_y
        self.test_x = test_x
        
        self.col_size = len(self.train_x.columns)

    def train(self,model_name=None, n_splits=2, n_estimator=3000, early_stopping_rounds=300,
              categorical_feature=[], random_state=42, verbose=1, test_size=0.3,):
        
        split_train_x, split_val_x, split_train_y, split_val_y = train_test_split(self.train_x, self.train_y, test_size=test_size, shuffle=True, random_state=random_state)
        
        if model_name == "CatBoostClassifier":
            self.model = CatBoostClassifier(n_estimators=n_estimator, 
                                            random_state=random_state,
                                            eval_metric="TotalF1",
                                            cat_features=categorical_feature
                                            )

            self.model.fit(split_train_x,split_train_y,
                            eval_set=[(split_val_x,split_val_y)],
                            early_stopping_rounds=early_stopping_rounds,
                            verbose=verbose,
                            )
            
        elif model_name =="LGBMClassifier":
            self.model = LGBMClassifier(n_estimators=n_estimator, 
                                        random_state=random_state,
                                        )
            self.model.fit(split_train_x,split_train_y,
                            eval_set=[(split_val_x,split_val_y)],        
                            early_stopping_rounds=early_stopping_rounds,
                            verbose=verbose,
                            categorical_feature=categorical_feature,
                            )
        elif model_name=="XGBClassifier":
            self.model = XGBClassifier(n_estimators=n_estimator, 
                                  random_state=random_state)
            self.model.fit(split_train_x,split_train_y,
                      eval_set=[(split_val_x,split_val_y)], 
                      early_stopping_rounds=early_stopping_rounds,
                      verbose=verbose)

    def train_dnn(self,dropout=0.2,activefunc="relu",epoch=2000,batch_size=40,patience=200,validation_split=0.2,verbose=1):
        # ---------------모델 구조 생성---------------
        self.model = tf.keras.Sequential()  

        self.model.add(layers.Dense(60, input_shape=(self.col_size,)))  
        self.model.add(layers.Activation(activefunc))  
        self.model.add(layers.Dropout(dropout))        

        self.model.add(layers.Dense(30))
        self.model.add(layers.Activation(activefunc))  
        self.model.add(layers.Dropout(dropout))    

        self.model.add(layers.Dense(3))
        self.model.add(layers.Activation('softmax')) 
        
        # ---------------모델 구축하기---------------
        self.model.compile(
            loss='categorical_crossentropy',  #다중 교차엔트로피
            optimizer="adam",   #최적화 기법 중 하나
            metrics=['accuracy'])  #정확도 측정
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
        # ---------------모델 학습---------------
        self.hist = self.model.fit(self.train_x, self.train_y, batch_size=batch_size, epochs=epoch, 
                                   validation_split=validation_split, verbose=verbose, callbacks = [callback])
        
    def predict(self):
        self.sample_submission["class"] = self.model.predict(self.test_x)
        return self.sample_submission
    
    def predict_proba(self):
        return self.model.predict_proba(self.test_x)

    def predict_dnn(self):
        def to_alpha(a):
            if a==0:
                return "A"
            elif a==1:
                return "B"
            elif a==2:
                return "C"

        self.sample_submission["class"] = np.argmax(self.model.predict(self.test_x),axis=1)
        self.sample_submission["class"] = self.sample_submission["class"].apply(to_alpha)

        return self.sample_submission
    def see_feature_importance(self,model,train_x,test_x):
        '''
        input : model과 model을 학습한 train_x
        output : 선택한 중요도 이상의 columns들을 가진 train_x, test_x로 데이터 재세팅
        '''
        pd.set_option('display.max_rows', None)
        ft_importance_values = model.feature_importances_
        ft_series = pd.Series(ft_importance_values, index = train_x.columns)
        df_desc = pd.DataFrame(ft_series.sort_values(ascending=False)).reset_index()
        display(df_desc)
        
        # [그래프 그리기]
        df = pd.DataFrame(ft_series.sort_values(ascending=True)).reset_index()
        fig = plt.figure(figsize=(30,20))
        plt.barh(df["index"],df[0])
        
        # 값 그래프 옆에 보이기
        for i,v in enumerate(df[0]):
            plt.text(v,i,str(round(v,6)))
        plt.show()

        cut = float(input("몇 이상부터 넣을지?"))

        df_cutted = df_desc.loc[df_desc[0]>=cut]
        display(df_cutted)
        df_cutted_list = list(df_cutted["index"])

        train_x_copy = train_x.copy()
        test_x_copy = test_x.copy()

        for col in train_x.columns:
            if col not in df_cutted_list:
                train_x_copy = train_x_copy.drop([col],axis=1)
                test_x_copy = test_x_copy.drop([col],axis=1)

        return train_x_copy,test_x_copy

    def train_dnn_v2(self,dropout=0.2,activefunc="relu",epoch=2000,batch_size=40,patience=200,validation_split=0.2,verbose=1):
        # ---------------모델 구조 생성---------------
        self.model = tf.keras.Sequential()  

        self.model.add(layers.Dense(120, input_shape=(self.col_size,)))  
        self.model.add(layers.Activation(activefunc))  
        self.model.add(layers.Dropout(dropout))        

        self.model.add(layers.Dense(60))       
        self.model.add(layers.Activation(activefunc))
        self.model.add(layers.Dropout(dropout))

        self.model.add(layers.Dense(30))       
        self.model.add(layers.Activation(activefunc))
        self.model.add(layers.Dropout(dropout))

        self.model.add(layers.Dense(3))
        self.model.add(layers.Activation('softmax')) 
        
        # ---------------모델 구축하기---------------
        self.model.compile(
            loss='categorical_crossentropy',  #다중 교차엔트로피
            optimizer="rmsprop",   #최적화 기법 중 하나
            metrics=["accuracy",tensorflow_addons.metrics.F1Score(num_classes=3, average='macro')])  #정확도 측정
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', patience=patience)
        # ---------------모델 학습---------------
        self.hist = self.model.fit(self.train_x, self.train_y, batch_size=batch_size, epochs=epoch, 
                                   validation_split=validation_split, verbose=verbose, callbacks = [callback])

In [ ]:
class MyModelCV:
    train_x = None
    train_y = None
    test_x = None
    
    model_type = None     
    model = None
    categorical_cols = None
    sample_submission = pd.read_csv(f"{data_folder}/sample_submission.csv")

    def __init__(self,train_x,train_y,test_x):
        self.train_x = train_x
        self.train_y = train_y
        self.test_x = test_x
        
    def train(self,model_name=None, n_estimator=3000, early_stopping_rounds=300,
              categorical_feature=[], random_state=42, verbose=1, test_size=0.3,):
        
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        data_args = (train_x,)

        score = 0
        for i,(tri, vai) in enumerate(cv.split(*data_args)):
            print("="*30,i,"="*30)
            
            if model_name == "CatBoostClassifier":
                model = CatBoostClassifier(n_estimators=n_estimator, 
                                                random_state=random_state,
                                                eval_metric="TotalF1",
                                                cat_features=categorical_feature
                                                )

                model.fit(train_x.iloc[tri],train_y.iloc[tri],
                               eval_set=[(train_x.iloc[vai],train_y.iloc[vai])],
                               early_stopping_rounds=early_stopping_rounds,
                               verbose=verbose,
                               )
            elif model_name =="LGBMClassifier":
                self.model = LGBMClassifier(n_estimators=n_estimator, 
                                            random_state=random_state)
                self.model.fit(train_x.iloc[tri],train_y.iloc[tri],
                               eval_set=[(train_x.iloc[vai],train_y.iloc[vai])],        
                               early_stopping_rounds=early_stopping_rounds,
                               eval_metric="F1",
                               verbose=verbose,
                               categorical_feature=categorical_feature,
                               )
                
            score += model.get_best_score()["validation"]["TotalF1"]
            
        score = score/n_splits
        
        return score

    def predict(self):
        self.sample_submission["class"] = self.model.predict(self.test_x)
        return self.sample_submission

In [ ]:
version = 8

if version==1:
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    params = {"model_name" : "CatBoostClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(preprocessor.snp_cols.keys()),
              "test_size":0.3,
    }

    my_model = MyModel(train_x,train_y,test_x)
    my_model.train(**params)
    my_sub = my_model.predict()
elif version==2:
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    params = {"model_name" : "LGBMClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(preprocessor.snp_cols.keys()),
              "test_size":0.3,
    }
    my_model = MyModel(train_x,train_y,test_x)
    my_model.train(**params)
    my_sub = my_model.predict()
elif version==3:
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    params = {"model_name" : "XGBClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(preprocessor.snp_cols.keys()),
              "test_size":0.3,              
    }
    my_model = MyModel(train_x,train_y,test_x)
    my_model.train(**params)
    my_sub = my_model.predict()
elif version==4: #11.csv
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x
    train_x = preprocessor.nominal_onehot(train_x,list(preprocessor.snp_cols.keys())+["trait"])
    test_x = preprocessor.nominal_onehot(test_x,list(preprocessor.snp_cols.keys())+["trait"])
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn()
elif version==5:
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    
    params = {"model_name" : "CatBoostClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(preprocessor.snp_cols.keys()),
              "test_size":0.3,
    }
    cat_model = MyModel(train_x,train_y,test_x)
    cat_model.train(**params)
    prob_array = np.array(cat_model.predict_proba())

    params = {"model_name" : "XGBClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(preprocessor.snp_cols.keys()),
              "test_size":0.3,              
    }
    xgb_model = MyModel(train_x,train_y,test_x)
    xgb_model.train(**params)
    prob_array += np.array(xgb_model.predict_proba())

    params = {"model_name" : "LGBMClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(preprocessor.snp_cols.keys()),
              "test_size":0.3,
    }
    lgbm_model = MyModel(train_x,train_y,test_x)
    lgbm_model.train(**params)
    prob_array += np.array(lgbm_model.predict_proba())

    def to_alpha(a):
        if a==0:
            return "A"
        elif a==1:
            return "B"
        elif a==2:
            return "C"
    sample_submission["class"] = np.argmax(prob_array,axis=1)
    sample_submission["class"] = sample_submission["class"].apply(to_alpha)
    my_sub = sample_submission
elif version==6: # 12.csv
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x,encoder_dict = preprocessor.do_label_encoding(train_x,preprocessor.snp_cols)
    test_x = preprocessor.labeling(test_x,encoder_dict)

    train_x = preprocessor.nominal_onehot(train_x,list(preprocessor.snp_cols.keys())+["trait"])
    test_x = preprocessor.nominal_onehot(test_x,list(preprocessor.snp_cols.keys())+["trait"])
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==7: 
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x
    my_model = MyModel(train_x,train_y,test_x)
    train_y_onehot = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])
    params = {"model_name" : "CatBoostClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(preprocessor.snp_cols.keys()),
              "test_size":0.3,
    }
    my_model.train(**params)
    pd.set_option('display.max_rows', None)

    train_x_select, test_x_select = my_model.see_feature_importance(my_model.model,train_x,test_x)

    train_x_select = preprocessor.nominal_onehot(train_x_select,list(train_x_select.columns))
    test_x_select = preprocessor.nominal_onehot(test_x_select,list(test_x_select.columns))

    dnn_model = MyModel(train_x_select,train_y_onehot,test_x_select)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":200,
              "validation_split":0.2
              }
    dnn_model.train_dnn(**params)
    my_sub = dnn_model.predict_dnn()   
elif version==8: #11.csv
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)
    
    train_x = preprocessor.nominal_onehot(train_x,list(train_x.columns))
    test_x = preprocessor.nominal_onehot(test_x,list(test_x.columns))
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2,
              "verbose":1,
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==9:
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)
    
    label_encoder_dict = {col:None for col in list(train_x.columns)}
    train_x,encoder_dict = preprocessor.do_label_encoding(train_x,label_encoder_dict)
    test_x = preprocessor.labeling(test_x,encoder_dict)

    params = {"model_name" : "CatBoostClassifier",
              "n_estimator": 3000, # epoch
              "early_stopping_rounds" : 200, 
              "categorical_feature": list(label_encoder_dict.keys()),
              "test_size":0.3,
              "verbose":1,
    }

    my_model = MyModel(train_x,train_y,test_x)
    my_model.train(**params)
    my_sub = my_model.predict()
elif version==10:
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)
    
    train_x = preprocessor.nominal_onehot(train_x,list(train_x.columns))
    test_x = preprocessor.nominal_onehot(test_x,list(test_x.columns))
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    train_x = preprocessor.add_order(train_x)
    test_x = preprocessor.add_order(test_x)

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2,
              "verbose":1,
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==11: 
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)
    
    train_x = preprocessor.nominal_onehot(train_x,list(train_x.columns))
    test_x = preprocessor.nominal_onehot(test_x,list(test_x.columns))
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":6000,
              "batch_size":10,
              "patience":500,
              "validation_split":0.15,
              "verbose":1,
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==12: 
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.same_col(train_x)
    test_x = preprocessor.same_col(test_x)

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)

    onehot_cols = [train_x.columns[0]]+list(train_x.columns[16:])
    train_x = preprocessor.nominal_onehot(train_x, onehot_cols)
    test_x = preprocessor.nominal_onehot(test_x, onehot_cols)
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])
    
    
    assert all(np.array(train_x.columns)==np.array(test_x.columns))

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2,
              "verbose":1,
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==13: 
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.same_col(train_x)
    test_x = preprocessor.same_col(test_x)
    
    train_x = train_x.drop(columns=Preprocess.snp_cols.keys())
    test_x = test_x.drop(columns=Preprocess.snp_cols.keys())

    onehot_cols = ["trait"]
    train_x = preprocessor.nominal_onehot(train_x, onehot_cols)
    test_x = preprocessor.nominal_onehot(test_x, onehot_cols)
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])
        
    assert all(np.array(train_x.columns)==np.array(test_x.columns))

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2,
              "verbose":1,
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==15: #11.csv
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)
    
    train_x = preprocessor.nominal_onehot(train_x,list(train_x.columns))
    test_x = preprocessor.nominal_onehot(test_x,list(test_x.columns))
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2,
              "verbose":1,
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==16:
    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)
    
    train_x = preprocessor.nominal_onehot(train_x,list(train_x.columns))
    test_x = preprocessor.nominal_onehot(test_x,list(test_x.columns))
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
              "activefunc":"relu",
              "epoch":3000,
              "batch_size":20,
              "patience":300,
              "validation_split":0.2,
              "verbose":1,
              }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 
elif version==17:
    def run_PCA(X, num_components):
        pca = sklearn.decomposition.PCA(n_components=num_components)
        pca.fit(X)
        pca_array = pca.transform(X)
        return pca, pca_array

    preprocessor = Preprocess(train,test)
    train_x,train_y,test_x = preprocessor.train_x,preprocessor.train_y,preprocessor.test_x

    train_x = preprocessor.split_alpha(train_x,preprocessor.snp_cols)
    test_x = preprocessor.split_alpha(test_x,preprocessor.snp_cols)

    label_encoder_dict = {col:None for col in list(train_x.columns)}
    train_x,encoder_dict = preprocessor.do_label_encoding(train_x,label_encoder_dict)
    test_x = preprocessor.labeling(test_x,encoder_dict)

    # print(train_x,test_x)
    pca, train_x_array = run_PCA(train_x,8)
    test_x_array = pca.transform(test_x)

    train_x = pd.DataFrame(train_x_array)
    test_x = pd.DataFrame(test_x_array)
    train_y = preprocessor.nominal_onehot(pd.DataFrame(preprocessor.train_y), ["class"])

    my_model = MyModel(train_x,train_y,test_x)
    params = {"dropout":0.2,
            "activefunc":"relu",
            "epoch":3000,
            "batch_size":20,
            "patience":300,
            "validation_split":0.2,
            "verbose":1,
                }
    my_model.train_dnn(**params)
    my_sub = my_model.predict_dnn() 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ft_series = pd.Series(ft_importance_values, index = train_x.columns)
df_desc = pd.DataFrame(ft_series.sort_values(ascending=False)).reset_index()
display(df_desc)

# [그래프 그리기]
df = pd.DataFrame(ft_series.sort_values(ascending=True)).reset_index()
fig = plt.figure(figsize=(30,20))
plt.barh(df["index"],df[0])

# 값 그래프 옆에 보이기
for i,v in enumerate(df[0]):
    plt.text(v,i,str(round(v,6)))
plt.show()

cut = float(input("몇 이상부터 넣을지?"))

df_cutted = df_desc.loc[df_desc[0]>=cut]
display(df_cutted)
df_cutted_list = list(df_cutted["index"])

train_x_copy = train_x.copy()
test_x_copy = test_x.copy()

for col in train_x.columns:
    if col not in df_cutted_list:
        train_x_copy = train_x_copy.drop([col],axis=1)
        test_x_copy = test_x_copy.drop([col],axis=1)

In [ ]:
# 제출용 (content에 dacon_submit_api넣어야한다 )
!pip install dacon_submit_api-0.0.4-py3-none-any.whl
from dacon_submit_api import dacon_submit_api 

# 제출파일 만들기
def get_file_name():
    def returnNum(s):
        return int(s[:-4])

    files = os.listdir(path+"/sub")
    try:
        new_file_num = max(list(map(returnNum,files)))+1
        file_name = str(new_file_num)+".csv"
    except:
        file_name = "1.csv"
    return file_name 
    
def submission():
    file_name = get_file_name()
    print(file_name)
    my_sub.to_csv(path+"/sub/"+file_name,index=False)

    # 제출
    sub_memo = f"version : {version}" # !!!!!!메모!!!!!!
    path_for_sub = path+"/sub/"+file_name
    token = "??"
    my_id = "??"
    my_team = "씩씩한오리너구리"

    result = dacon_submit_api.post_submission_file(
        path_for_sub, 
        token, 
        my_id, 
        my_team, 
        sub_memo )
    
    return result

submission()    